In [3]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [4]:
# Paths
fall_folder = 'fall'
not_fall_folder = 'not-fall'

# Parameters
IMG_SIZE = 224

In [5]:
# Load Data
def load_images(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        img = cv2.imread(filepath)
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)


In [6]:
# Load fall and not-fall images
fall_images, fall_labels = load_images(fall_folder, 1)
not_fall_images, not_fall_labels = load_images(not_fall_folder, 0)

# Combine data
X = np.vstack((fall_images, not_fall_images))
y = np.hstack((fall_labels, not_fall_labels))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Feature Extraction with ResNet50
resnet = ResNet50(weights='imagenet', include_top=False, pooling='avg')

def extract_features(images):
    images = preprocess_input(images)
    return resnet.predict(images)

X_train_features = extract_features(X_train)
X_test_features = extract_features(X_test)

15/15 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step


In [8]:
clf = SVC(kernel='linear', probability=True)
clf.fit(X_train_features, y_train)

# Predict and Evaluate
y_pred = clf.predict(X_test_features)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

Model Accuracy: 93.91%


In [9]:
# Save Model
import joblib
joblib.dump(clf, "fall_detection_model.pkl")

# Save ResNet Feature Extractor
resnet.save("resnet_feature_extractor.h5")